In [4]:
import pandas as pd

In [5]:
# Scrapped in 07 september 
df = pd.read_csv('07 September, 2020_listings.csv')
df.shape

(45756, 74)

### Helper function 

These helper functions are there to help the feature data

In [157]:
# to change the numeric type strigs to float values
def toNumeric(a):
    a = a.replace('$', '')
    a = a.replace(',', '')
    return float(a)

# to change object type to strings
def toString(a):
    return str(a)

# to remove the percentage and return as  
def fpercentage(a):
    if str(a) == 'nan':
        return -1
    else:
        a = str(a)
        a = a.replace('%', '')
        a = int(a)
        return a/100

# to change into boolean 
def toBoolean(a):
    if str(a) == 'f':
        return False
    if str(a) == 't':
        return True

# limit the about host string
def fhostabout(a):
    a = str(a)
    a = a[:999]
    return a

# limit the neighboorhood string
def hostneigh(a):
    a = str(a)
    a = a[:50]
    return a

# clean the description string
def cleanText(a):
    a = str(a)
    a = a.replace('</b>', '')
    a = a.replace('<b>', '')
    a = a.replace('<br />', '')
    a = a.replace("\n", '')
    a = a.replace("\r", '')
    return a

# clean nan value to -1
def cbed(a):
    if str(a) == 'nan':
        return -1.0

    else:
        a = str(a)
        a = a.replace(' ', '')
        return float(a)

In [159]:
# Using above helper function clean the each features

df['host_since'] = pd.to_datetime(df['host_since'])
df['price'] = df['price'].apply(lambda x: toNumeric(x))
df['host_name'] = df['host_name'].apply(lambda x: toString(x))
df['host_response_rate'] = df['host_response_rate'].apply(lambda x: fpercentage(x))
df['host_identity_verified'] = df['host_identity_verified'].apply(lambda x: toBoolean(x))
df['host_has_profile_pic'] = df['host_has_profile_pic'].apply(lambda x: toBoolean(x))
df['host_location'] = df['host_location'].apply(lambda x: hostneigh(x))
df['host_about'] = df['host_about'].apply(lambda x: fhostabout(x))
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(lambda x: fpercentage(x))
df['host_is_superhost'] = df['host_is_superhost'].apply(lambda x: toBoolean(x))
df['property_type'] = df['property_type'].apply(lambda x: toString(x))
df['host_neighbourhood'] = df['host_neighbourhood'].apply(lambda x: hostneigh(x))
df['room_type'] = df['room_type'].apply(lambda x: toString(x))
df['listing_url'] = df['listing_url'].apply(lambda x: toString(x))
df['picture_url'] = df['picture_url'].apply(lambda x: toString(x))
df['description'] = df['description'].apply(lambda x: toString(x))
df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].apply(lambda x: toString(x))
df['amenities'] = df['amenities'].apply(lambda x: toString(x))
df['instant_bookable'] = df['instant_bookable'].apply(lambda x: toBoolean(x))
df['description'] = df['description'].apply(lambda x: cleanText(x))
df['host_about'] = df['host_about'].apply(lambda x: cleanText(x))

### We choose the following features

#### TO DO: Explain why the following features are choosen:

#### Hint: Normalizaiton

Designing a Database from above features:

    About Host
        1.host_id
        2.host_name
        3.host_since
        4.host_response_rate
        5.host_identity_verified
        6.host_location
        7.host_about - description of host
        8.host_response_rate

        10.host_is_superhost
        11.host_neighbourhood
        12.host_listing_count
        13.host_has_profile_pic
    
    
    Aboout Listing
        1.id
        2.listing_url
        3.picture_url
        4.description
        5.neighbourhood_cleansed
        6.latitude
        7.longitude
        8.property_type
        9.room_type
        10.accommodates
        11.bedrooms
        12.beds
        13.amenities
        14.price
        15.minimum_nights
        16.review_scores_rating
        17.review_scores_accuracy
        18.review_scores_value
        19.instant_bookable
        20.reviews_per_month
        

In [166]:
relevant = ['host_id', 'host_name','host_since', 'host_response_rate', 'host_identity_verified', 'host_has_profile_pic', 'host_location', 
           'host_about', 'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
           'calculated_host_listings_count', 'id', 'listing_url', 'picture_url', 'description', 
            'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
           'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'review_scores_rating', 'review_scores_accuracy',
           'review_scores_value', 'instant_bookable', 'reviews_per_month']

#### Host table 

In [167]:
host = ['host_id', 'host_name','host_since', 'host_response_rate', 'host_identity_verified', 'host_has_profile_pic', 'host_location', 
           'host_about', 'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood',
           'calculated_host_listings_count']

#### Listing table

In [168]:
listing = ['id', 'listing_url', 'picture_url', 'description', 
            'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
           'bedrooms', 'beds', 'price', 'minimum_nights', 'review_scores_rating', 'review_scores_accuracy',
           'review_scores_value', 'instant_bookable', 'reviews_per_month', 'host_id']

In [170]:
# store all the features than have location/address data to create address table

hl = df.host_location.values
hn = df.host_neighbourhood.values
nc = df.neighbourhood_cleansed.values

In [171]:
# create a list of all the location/address

locations = []
for i in hl:
    if i not in locations:
        locations.append(i)
print(len(locations))  

for j in hn:
    if j not in locations:
        locations.append(j)
print(len(locations))

for k in nc:
    if k not in locations:
        locations.append(k)
print(len(locations))

1610
2069
2111


In [173]:
# create a locaiton dictionary where locaiton is key and id is the value
# using location as key will help us identify the id for the host and listings efficiently

locations_dict = {}
for i in range(0, len(locations)):
    a = locations[i]
    locations_dict[a] = i

In [174]:
# store the key and values to create a address table

address = locations_dict.keys()
address_id = locations_dict.values()

In [175]:
# Functions to resolve the id of the host locaiton and neighborhood that would be used 
# as the foreign key in the host table

def resolveHostAddressId(df):
    host_location_id = []
    host_neighbourhood_id = []
    for i in range(df.shape[0]):
        a = df.host_location.iloc[i]
        b = df.host_neighbourhood.iloc[i]
        value_a = locations_dict[a]
        value_b = locations_dict[b]
        host_location_id.append(value_a)
        host_neighbourhood_id.append(value_b)  
    return host_location_id, host_neighbourhood_id

# Functions to resolve the id of the listing neighborhood that would be as the foreign key in listing table

def resolveListingAddressId(df):
    location_id = []
    for i in range(df.shape[0]):
        c = df.neighbourhood_cleansed.iloc[i]
        value_c = locations_dict[c]
        location_id.append(value_c)
    return location_id

#### Creating a address dataframe and saving it as a csv file

In [176]:
dfa = pd.DataFrame(address_id)
dfa['address'] = address
dfa.rename(columns={0:'address_id'})
# dfa.to_csv('address_airbnb.csv', index=False)

,address_id,address
0,0,"New York, New York, United States"
1,1,US
2,2,"Brooklyn, New York, United States"
3,3,"Berkeley, California, United States"
4,4,"Sarasota, Florida, United States"
...,...,...
2106,2106,Breezy Point
2107,2107,Castle Hill
2108,2108,Willowbrook
2109,2109,Little Neck


#### Creating a host and lisitng dataframe

In [177]:
dfh = pd.DataFrame(df, columns = host)
dfl = pd.DataFrame(df, columns = listing)

In [178]:
# calling the id resolve funciton
host_location_id, host_neighbourhood_id =  resolveHostAddressId(dfh)
neighbourhood_cleansed_id = resolveListingAddressId(dfl)

In [179]:
# storing the id as the feature in the dataframe
dfh['host_location_id'] = host_location_id
dfh['host_neighbourhood_id'] = host_neighbourhood_id
dfl['neighbourhood_cleansed_id'] = neighbourhood_cleansed_id

In [180]:
# dropping the address features
dfh = dfh.drop(columns = ['host_location', 'host_neighbourhood'])
dfl = dfl.drop(columns = ['neighbourhood_cleansed'])

In [181]:
print(dfh.shape)
print(dfl.shape)

(45756, 12)
(45756, 20)


In [182]:
# since hostid is the primary id of the host table let's drop all the duplicates
dfh = dfh.drop_duplicates()

In [183]:
dfh.shape

(34826, 12)

In [184]:
# dropping those host who don't have host_since data (only 12)
dfh = dfh.dropna(subset=['host_since'])

In [185]:
dfh.shape

(34814, 12)

In [186]:
dfl.shape

(45756, 20)

In [187]:
# further cleanning of the data with helper funcitons to change the datatype into float
dfl['bedrooms'] = dfl['bedrooms'].apply(lambda x: cbed(x))
dfl['beds'] = dfl['beds'].apply(lambda x: cbed(x))
dfl['review_scores_rating'] = dfl['review_scores_rating'].apply(lambda x: cbed(x))
dfl['review_scores_accuracy'] = dfl['review_scores_accuracy'].apply(lambda x: cbed(x))
dfl['review_scores_value'] = dfl['review_scores_value'].apply(lambda x: cbed(x))
dfl['reviews_per_month'] = dfl['reviews_per_month'].apply(lambda x: cbed(x))

#### Save the cleaned file in csv 

In [191]:
dfh.to_csv('host-airbnb.csv', index=False)
dfl.to_csv('listing-airbnb.csv', index=False)

In [189]:
# missed = pd.DataFrame()
# missed['column'] = dfr.columns

# missed['percent'] = [
#     round(100* dfr[col].isnull().sum() / len(dfr), 2) for col in dfr.columns
# ]
# missed = missed[missed['percent'] >0].sort_values('percent')

# fig = px.bar(
#     missed, 
#     x='percent',
#     y="column", 
#     orientation='h', 
#     title='Missed values percent for every column (percent > 0)', 
#     height=1300, 
#     width=800
# )

# fig.show()